In [1]:
import pandas as pd
import json
from os import listdir
from os.path import isfile, join
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
from scipy.stats import gaussian_kde
from sklearn.ensemble import IsolationForest
from PIL import Image
from matplotlib.ticker import FuncFormatter
from scipy.spatial import KDTree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import csv

In [ ]:
dir_path = "./json_files"
all_json_files = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f)) and f.endswith(".json")]

# List to hold DataFrames
df_list = []
df_2list = []

# Process each JSON file (each file represents a game)
for file_name in all_json_files:
    with open(file_name) as f:
        data = json.load(f)
        
    player_data = {}

    # Loop through all telemetry events
    for event in data['rawTelemetry']:
        if 'character' in event:
            player_name = event['character']['name']
            current_z = event['character']['location']['z']

            # Initialize player data and tracking variables if necessary
            if player_name not in player_data:
                player_data[player_name] = {
                    'coordinates': [],
                    'record': False,
                    'last_z': None,
                    'prev_event': None
                }

            pdata = player_data[player_name]
            if pdata['last_z'] is not None and pdata['last_z'] > 150000 and current_z <= 150000:
                if pdata['prev_event'] is not None:
                    pdata['coordinates'].append(pdata['prev_event'])
                pdata['record'] = True

            if pdata['record']:
                if pdata['last_z'] is not None and abs(pdata['last_z'] - current_z) < 999:
                    pdata['record'] = False
                if pdata['record']:
                    pdata['coordinates'].append(event['character']['location'])

            pdata['last_z'] = current_z
            pdata['prev_event'] = event['character']['location']

    # Convert player_data to DataFrames for this game
    for player_name, pdata in player_data.items():
        coordinates = pdata['coordinates']
        if len(coordinates) >= 8:
            coordinates = coordinates[:7]
        if len(coordinates) in [5, 6, 7]:
            df = pd.DataFrame(coordinates[:-1])
            df_2 = pd.DataFrame([coordinates[-1]])
            df['player_name'] = player_name  # Add player name column
            df_list.append(df)
            df_2['player_name'] = player_name  # Add player name column
            df_2list.append(df_2)

# Concatenate and save the DataFrames
all_data_df = pd.concat(df_list, ignore_index=True)
all_data_df.to_csv('all_player_coordinates_test.csv', index=False)

all_data_df_2 = pd.concat(df_2list, ignore_index=True)
all_data_df_2.to_csv('all_player_coordinates_landing_test.csv', index=False)